In [11]:
import sqlalchemy

In [13]:
%sql sqlite:///Analysis.db

'Connected: @Analysis.db'

In [14]:
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
import pandas
import csv, sqlite3

con = sqlite3.connect("Analysis.db")
cur = con.cursor()

In [5]:
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

533

In [6]:
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

566

In [7]:
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

78

In [15]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///Analysis.db
Done.


name
CHICAGO_CRIME_DATA
CHICAGO_PUBLIC_SCHOOLS_DATA
CENSUS_DATA


1. Find the total number of crimes recorded in the CRIME table 

In [16]:
%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000;

 * sqlite:///Analysis.db
Done.


COMMUNITY_AREA_NAME
West Garfield Park
South Lawndale
Fuller Park
Riverdale


 2. List community areas with per capita income less than 11000

In [17]:
%%sql
select
 community_area_number,
 community_area_name,
 per_capita_income
from
  CENSUS_DATA
where
  per_capita_income < 11000

 * sqlite:///Analysis.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
26.0,West Garfield Park,10934
30.0,South Lawndale,10402
37.0,Fuller Park,10432
54.0,Riverdale,8201


3. List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [18]:
%%sql
select
  id,
  case_number, 
  primary_type,
  description
from
  CHICAGO_CRIME_DATA
where
  description like '%minor%'

 * sqlite:///Analysis.db
Done.


ID,CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
3987219,HL266884,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR
3266814,HK238408,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR


4. List all kidnapping crimes involving a child?

In [19]:
%%sql
select
  id,
  case_number,
  primary_type,
  description
from
  CHICAGO_CRIME_DATA
where
  primary_type like '%kidnapping%'
  and
  description like '%child%'

 * sqlite:///Analysis.db
Done.


ID,CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
5276766,HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


5. What kinds of crimes were recorded at schools?


In [20]:
%%sql
select
  distinct primary_type, 
  /* description ,*/
  location_description
from
  CHICAGO_CRIME_DATA
where
  location_description like '%school%'
order by
  primary_type

 * sqlite:///Analysis.db
Done.


PRIMARY_TYPE,LOCATION_DESCRIPTION
ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
PUBLIC PEACE VIOLATION,"SCHOOL, PRIVATE, BUILDING"
PUBLIC PEACE VIOLATION,"SCHOOL, PUBLIC, BUILDING"


6. List the average safety score for each type of school.

In [21]:
%%sql
select
  "Elementary, Middle, or High School",
  avg(safety_score)
from
  CHICAGO_PUBLIC_SCHOOLS_DATA
group by  
  "Elementary, Middle, or High School"

 * sqlite:///Analysis.db
Done.


"Elementary, Middle, or High School",avg(safety_score)
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


7. List 5 community areas with highest % of households below poverty line

In [22]:
%%sql
select
  community_area_name,
  max(percent_households_below_poverty) as highest_percent_households_below_poverty
from
  census_data
where
  percent_households_below_poverty is not null
group by 
  community_area_name
order by 
  highest_percent_households_below_poverty desc
limit 5

 * sqlite:///Analysis.db
Done.


COMMUNITY_AREA_NAME,highest_percent_households_below_poverty
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


8. Which community area is most crime prone?

In [23]:
%%sql
select
  community_area_number,
  count(community_area_number) as CrimeNumber
from
  CHICAGO_CRIME_DATA
where
  community_area_number is not null
group by  
  community_area_number
order by 
  crimenumber desc
limit 1

 * sqlite:///Analysis.db
Done.


COMMUNITY_AREA_NUMBER,CrimeNumber
25.0,43


9. Use a sub-query to find the name of the community area with highest hardship index

In [24]:
%%sql
select
  community_area_number,
  community_area_name,
  hardship_index
from
  census_data
where
  hardship_index = (
    select max(hardship_index)
    from census_data
  )

 * sqlite:///Analysis.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,HARDSHIP_INDEX
54.0,Riverdale,98.0



 10. Use a sub-query to determine the Community Area Name with most number of crimes?

In [25]:
%%sql
select
  distinct ccd.community_area_number, cd.community_area_name
from
  CHICAGO_CRIME_DATA ccd
join 
  census_data cd
on  
  ccd.community_area_number = cd.community_area_number
where
  ccd.community_area_number = (
    select community_area_number
    from CHICAGO_CRIME_DATA
    where community_area_number is not null
    group by community_area_number
    order by count(community_area_number) desc 
    limit 1
  )

 * sqlite:///Analysis.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
25.0,Austin
